# Machine Learning, a Microscope for Genomics
## (A practical demonstration in Python)

The goal of this notebook is to demonstrate how a very simple machine learning algorithm can provide insights into a relatively complicated genomic dataset. Specifically, we will use **Naive Bayes** to predict whether or not a gene is active based on histone modifications near its **Transcriptional Start Site (TSS)**. 

<img src="./image_for_notebook/image_for_notebook.001.png">

Because these datasets are so large (there are over 20,000 protein coding genes in the human genome), going through them by hand and observing correlations with gene activity is not possible. However, machine learning allows us to find correlations and identify active genes. In my mind, this makes machine learning a 21st century tool for science in much the same way that a microscope helped researchers in the 19th and 20th centuries.

This tutorial uses data from [The Encode Project](https://www.encodeproject.org/) derived from a cell line called [K562](https://en.wikipedia.org/wiki/K562_cells), which are human leukemia cells. First, I downloaded [total RNA-seq](https://www.encodeproject.org/files/ENCFF553DDU/@@download/ENCFF553DDU.tsv) data to identify active and inactive genes. This will allow us to determine if our simple machine learning method works or not. And, in order to make predictions, I downloaded ChIP-seq data for the following five histone modifications: 
* [H3K27me3](https://www.encodeproject.org/files/ENCFF700RBU/@@download/ENCFF700RBU.bed.gz)
* [H3K36me3](https://www.encodeproject.org/files/ENCFF028HJF/@@download/ENCFF028HJF.bed.gz)
* [H3K4me1](https://www.encodeproject.org/files/ENCFF501DZN/@@download/ENCFF501DZN.bed.gz)
* [H3K4me3](https://www.encodeproject.org/files/ENCFF500EUY/@@download/ENCFF500EUY.bed.gz)
* [H3K9me3](https://www.encodeproject.org/files/ENCFF954LYH/@@download/ENCFF954LYH.bed.gz)

All datasets were aligned to [GRCh38](https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.26/), which is a version of the human genome curated by the [Genome Reference Consortium](https://www.ncbi.nlm.nih.gov/grc).

----

# Import the modules that will do all the work
The very first thing we do is load in a bunch of python modules. Python, itself, just gives us a basic programming language. These modules give us extra functionality to import the data, clean it up and format it, and then build and evaluate the Naive Bayes classifier. 

**NOTE:** You will need **Python 3** and have at least these versions for each of the following modules: 
- pandas >= 0.25.1
- numpy >= 1.17.2
- sklearn >= 0.22.1
 
If you installed **Python 3** with [Anaconda](https://www.anaconda.com/), you can check which version you have with the command: `conda list`. If, for example, your version of `scikit-learn` is older than 0.22.1, then the easiest thing to do is just update all of your **Anaconda** packages with the following command: `conda update --all`. However, if you only want to update `scikit-learn`, then you can run this command: `conda install scikit-learn=0.22.1`.

In [ ]:
import pandas as pd # load and manipulate data
import matplotlib.pyplot as plt # for drawing graphs
import seaborn as sns # for drawing graphs that look nice
from sklearn.model_selection import train_test_split # to split data into training and testing sets
from sklearn.naive_bayes import MultinomialNB # the Naive Bayes stuff
from sklearn.metrics import confusion_matrix # to create a confusion matrix
from sklearn.metrics import plot_confusion_matrix # to draw a confusion matrix

---

# Import the RNA-seq data
First, let's load in the RNA-seq data.

**NOTE:** When **pandas** (**pd**) reads in data, it returns a **DataFrame**, which is a lot like a spreadsheet. The data are organized in rows and columns and each row can contain a mixture of text and numbers. The standard variable name for a **DataFrame** is the initials **df**, and that is what we add as a suffix to our variables names so we can keep track of what is what:

In [ ]:
rnaseq_df = pd.read_table('rnaseq.tsv', sep='\t')

Now that we have loaded the data into a **DataFrame** called **rnaseq_df**, let's look at the first five rows using the `head()` function:

In [ ]:
## print the first 5 rows
rnaseq_df.head()

OK, so the first thing we see are `gene_ids` that are just numbers and not gene names. Depending on whether or not we also have the gene coordinates (chromosome number and position), this could be a problem. Looking at the other columns, I do not see gene coordinates so we have to figure out what these numbers mean and how we can translate them into gene names, and, ultimately, how those gene names can be associated with genomic coordinates.

Fortunately, a quick google search Google search on the terms "encode gene id to gene name" told me that if a `gene_id` is just a number, then it corresponds to **tRNA** and **pseudo_tRNA** sequences, and this is confirmed by the length of these genes (which are all less than 100 bp). 
I then read that if we scrolled down a bunch, we will see **ENSEMBL** gene ids like `ENSG00000000003.10`. So let's look and see if we can find the **ENSEMBL** gene ids. We'll do this by first determining how many genes are in the file using printing out the DataFrame's `shape`, which tells us how many rows and columns it has.

In [ ]:
rnaseq_df.shape

Since there are 58,540 rows in this file, we can pick a few in the middle to see if there have proper ENSEMBL gene ids.

In [ ]:
rnaseq_df.iloc[20000:20005] ## iloc[] allows us to specify which rows want to look at by number

Hooray!!! When we look at the rows in the middle of the DataFrame, we see proper ENSEMBL gene ids. We can use these to determine the **Transcription Start Sites (TSSs)** for each gene. Once we identify the TSSs, we can determine if the ChIP-seq data is enriched in those regions and use machine learning to determine if those enriched sites correlate with gene activity or not. However, first let's remove **tRNA** and **pseudo_tRNA** data, since we can not associate those numeric IDs with a corresponding TSS. We can do this by selecting all of the `gene_ids` that start with **ENSG** and putting them into a new DataFrame called `ensembl_gene_df`.

In [ ]:
ensembl_gene_df = rnaseq_df[rnaseq_df['gene_id'].str.contains('ENSG')].copy()
ensembl_gene_df.head()

**NOTE:** In the output from the `head()` function, we see that a single value for `gene_id` can map to many different (comma separated) values for `transcript_id(s)`. This means that we know the read counts on a gene level, but can not tell if one specific transcript gets more reads than another. Ideally, we would have read counts on a transcript level, but since this is not the case, we must and make the best of what we have. **SPOILER ALERT** Even with this simplification, **Naive Bayes** still performs pretty well!!

Now we can determine how many **tRNA** and **pseduo_tRNAs** we removed by doing a little math with the output from `shape`. Since, in this case, `shape` returns the number of row and columns in an array, we can access the row number as the first (index = 0) element in each array.

In [ ]:
rnaseq_df.shape[0] - ensembl_gene_df.shape[0]
## Alternatively, we can use the len() function (len = length) to do the same thing
# len(rnaseq_df) - len(ensembl_gene_df)

So we removed 720 **tRNA** and **pseduo_tRNAs**.

Now let's make our lives a little simpler by keeping the two columns we need `gene_id` and `posterior_mean_count`, and omitting all of the columns we do not need.

In [ ]:
ensembl_gene_df = ensembl_gene_df[['gene_id', 'posterior_mean_count']].copy()
ensembl_gene_df.head()

Lastly, we can make our life a lot easier later on if we rename the second column `posterior_mean_count` to just `count`.

In [ ]:
ensembl_gene_df.rename({'posterior_mean_count' : 'count'}, axis='columns', inplace=True)
ensembl_gene_df.head()

---

# Merging the RNA-seq data with Transcription Start Sites (TSSs)

Because `ensembl_gene_df` does not have the TSS locations, we have to download them from Ensembl's [BioMart](http://uswest.ensembl.org/biomart/martview/), import them and merge them with the data we have. We can get the data from BioMart by selecting the **Ensembl Genes Database** for **Human Genes (GRCh38)** and, from the **Attributes** section, selecting **Gene stable ID**, **Chromosome/scaffold name** and **Transcription start site (TSS)** with [this query](http://uswest.ensembl.org/biomart/martview/44be506488a1d112e1b393c5b4f15624?VIRTUALSCHEMANAME=default&ATTRIBUTES=hsapiens_gene_ensembl.default.feature_page.ensembl_gene_id|hsapiens_gene_ensembl.default.feature_page.ensembl_gene_id_version|hsapiens_gene_ensembl.default.feature_page.ensembl_transcript_id|hsapiens_gene_ensembl.default.feature_page.ensembl_transcript_id_version|hsapiens_gene_ensembl.default.feature_page.chromosome_name|hsapiens_gene_ensembl.default.feature_page.start_position|hsapiens_gene_ensembl.default.feature_page.end_position|hsapiens_gene_ensembl.default.feature_page.transcription_start_site&FILTERS=&VISIBLEPANEL=attributepanel). So now let's load in the file with the TSS coordinates and look at the first few rows.

In [ ]:
## NOTE: The file we are reading in is relatively large (25 mb) and complicated, so we set low_memory=False
tss_df = pd.read_table('ensembl_geneids_to_tss.txt.gz', sep='\t', low_memory=False)
tss_df.head()

Now, in order to make our lives, easier, let's give the columns we are interested in, `Gene stable ID`, `Chromosome/scaffold name` and `Transcription start site (TSS)` shorter names...

In [ ]:
tss_df.rename(columns={"Gene stable ID": "gene_id_stable", 
                       "Chromosome/scaffold name": "chr", 
                       "Transcription start site (TSS)": "tss"}, inplace=True)
tss_df.head()

...and let's make our lives a little simpler by just keeping the columns we are interested in and omitting the rest.

In [ ]:
tss_df = tss_df[['gene_id_stable', 'chr', 'tss']].copy()
tss_df.head()

**NOTE:** Because we downloaded **Gene stable IDs** from BioMart, we have to convert the gene IDs in `ensembl_gene_df` to stable gene ids. Stable gene IDs are the ENSG IDs that we have, but everything after the "dot" is excluded. For example, if we look at the first 5 gene IDs in `ensembl_gene_df` with `head()`...

In [ ]:
ensembl_gene_df['gene_id'].head()

...we see that each gene id ends in "dot some number". We need to remove the "dot some number", so we'll do this with something called a **Regular Expression**. **Regular Expressions** can be very cryptic and are beyond the scope of this notebook, however, just know that they are like a "search and replace" tool on steroids. In this case, we'll use a **Regular Expression** to extract the stable IDs and put them into a new variable called `gene_id_stable`.

In [ ]:
## NOTE: I found out while I was doing this that Ensembl starts most gene names with "ENSG", but it starts genes
## that are on the pseudo-autosomal region on the Y-chromosome with "ENSGR" because it allows them to add two 
## sets of coordinates for a single gene (one set of coordinates for the X chromosome and one set for the Y chromosome)
## to a GTF file (which is a file used by programs that count reads per gene).
## Anyway, because some gene ids start with 'ENSG' and others start with ENGSR, we need a relatively complicated
## regular expression
gene_id_stable = ensembl_gene_df['gene_id'].str.extract(pat='(ENSG[R]?\d*)')
gene_id_stable[0:4]

Now that we have the stable gene IDs, let's add them to `ensembl_gene_df` by creating a new column called `gene_id_stable`. 

In [ ]:
ensembl_gene_df['gene_id_stable'] = gene_id_stable
# NOTE: Alternatively, we could have created a new column with the following code:
# ensembl_gene_df = ensembl_gene_df.assign(gene_id_stable=gene_id_stable)
ensembl_gene_df.head()

Now we can merge `ensembl_gene_df` with `tss_df` based on `gene_id_stable` using the `merge()` function.

In [ ]:
count_and_tss = ensembl_gene_df.merge(tss_df, on='gene_id_stable')
count_and_tss.head()

Now, if look at the output from the last call to `head()`, we see that a single value for `gene_id` and `gene_id_stable` can map to multiple values in `tss`. We also saw this earlier when we first created the `ensembl_gene_df` DataFrame. Since we only have read counts per gene, and not per transcript, we have to just pick a single TSS per `gene_id`. This is not ideal, but like I mentioned before in a **SPOILER ALERT**, our model still performs pretty well.

In [ ]:
## This code groups the rows by gene_id_stable and then selects the first entry in each group
count_and_tss = count_and_tss.groupby('gene_id_stable').first()
count_and_tss.head()
## NOTE: This process converted the column gene_id_stable into row names (indexes),
## and that is why gene_id_stable is now bold and lower than the other column names.

Now, for each gene, we need to add columns for the ChIP-seq data. These columns will tell us how much ChIP-seq enrichment is within a 2kb window (1kb upstream and 1 kb downstream) around the TSS. Because the easiest way to identify ChIP-seq enrichment within a window is using a separate program called **bedTools**, we will have to export the data we have in [**BED** format](https://m.ensembl.org/info/website/upload/bed.htmlhttps://m.ensembl.org/info/website/upload/bed.html). In essence, that means reordering the columns.

In [ ]:
count_and_tss.index.name = 'gene_id_stable' ## first, let's make gene_id_stable just another column
count_and_tss.reset_index(inplace=True) ## this makes the row names (index) just numbers
tss_bed = count_and_tss[['chr', 'tss', 'tss', 'gene_id_stable']].copy() ## this selects the columns in BED format order
tss_bed.head()

Now we need to add the prefix **chr** to all of the chromosome numbers.

In [ ]:
tss_bed["chr"] = "chr" + tss_bed["chr"].astype(str)
tss_bed.head()

Now that we have the data in **BED** format, we can write it to a file and use **bedTools** to find ChIP-seq enrichment near each TSS.

In [ ]:
tss_bed.to_csv("tss.bed", sep='\t', header=False, index=False)

I then used "bedTools" to find ChIP-seq peaks within 1kb of the TSSs. These are the commands that I used:
```
bedtools window -a tss.bed -b h3k27me3.bed.gz -w 2000 -c > tss_h3k27me3.bed

bedtools window -a tss.bed -b h3k36me3.bed.gz -w 2000 -c > tss_h3k36me3.bed

bedtools window -a tss.bed -b h3k4me1.bed.gz -w 2000 -c > tss_h3k4me1.bed

bedtools window -a tss.bed -b h3k4me3.bed.gz -w 2000 -c > tss_h3k4me3.bed

bedtools window -a tss.bed -b h3k9me3.bed.gz -w 2000 -c > tss_h3k9me3.bed
```

Now we can load in the new files that show if the modified histones are enriched near the TSSs and merge them together. We will start by loading in `tss_h3k27me3.bed` and looking at the first 5 rows.

In [ ]:
tss_h3k27me3 = pd.read_table('tss_h3k27me3.bed', sep='\t', header=None)
tss_h3k27me3.head()

Since the new file does not have column names, we can add column names...

In [ ]:
tss_h3k27me3.columns=["chr", "start", "stop", "gene_id_stable", "h3k27me3"]
tss_h3k27me3.head()

...and, because we only need the last two columns (we need `gene_id_stable` to merge this file with `count_and_tss`, and we need `h3k27me3` because that has the number of enriched sites near the TSS), we can drop the first three columns.

In [ ]:
tss_h3k27me3.drop(columns=["chr", "start", "stop"], inplace=True)
tss_h3k27me3.head()

Now we can merge the number of enriched sites with `count_and_tss`...

In [ ]:
count_and_tss = count_and_tss.merge(tss_h3k27me3, on='gene_id_stable')
count_and_tss.head()

Now we just do the same thing for all of the other ChIP-seq files...

In [ ]:
tss_chipseq = pd.read_table('tss_h3k36me3.bed', sep='\t', header=None)
tss_chipseq.columns=["chr", "start", "stop", "gene_id_stable", "h3k36me3"]
tss_chipseq.drop(columns=["chr", "start", "stop"], inplace=True)
count_and_tss = count_and_tss.merge(tss_chipseq, on='gene_id_stable')
count_and_tss.head()

In [ ]:
tss_chipseq = pd.read_table('tss_h3k4me1.bed', sep='\t', header=None)
tss_chipseq.columns=["chr", "start", "stop", "gene_id_stable", "h3k4me1"]
tss_chipseq.drop(columns=["chr", "start", "stop"], inplace=True)
count_and_tss = count_and_tss.merge(tss_chipseq, on='gene_id_stable')
count_and_tss.head()

In [ ]:
tss_chipseq = pd.read_table('tss_h3k4me3.bed', sep='\t', header=None)
tss_chipseq.columns=["chr", "start", "stop", "gene_id_stable", "h3k4me3"]
tss_chipseq.drop(columns=["chr", "start", "stop"], inplace=True)
count_and_tss = count_and_tss.merge(tss_chipseq, on='gene_id_stable')
count_and_tss.head()

In [ ]:
tss_chipseq = pd.read_table('tss_h3k9me3.bed', sep='\t', header=None)
tss_chipseq.columns=["chr", "start", "stop", "gene_id_stable", "h3k9me3"]
tss_chipseq.drop(columns=["chr", "start", "stop"], inplace=True)
count_and_tss = count_and_tss.merge(tss_chipseq, on='gene_id_stable')
count_and_tss.head()

**BAM!!!** We merged the RNA-seq data with the ChIP-seq data.

----

# Classify genes as Active or Not Active

Now we need to classify genes as "active" or "not active". However, first, let's draw a boxplot of the values in `count` to see how the genes are spread out.

In [ ]:
g = sns.catplot(y='count', data=count_and_tss, kind="box")
g.fig.get_axes()[0].set_yscale("log") ## using the log() on the y-axis makes things easier to see

And we see that most genes have relative low read counts. We can also print out a summary of the values in `count` to see the quantiles.

In [ ]:
count_and_tss['count'].describe()

Confirming what we saw in the boxplot, most genes have 0 reads. So let's see what happens if we say that if a gene has > 0 reads, it is *active*, otherwise it is *inactive*. We can do that by creating a new column `active` and setting it to `True` when there is more than 1 read associated with that gene. Otherwise we will set the value to `False`. 

**NOTE:** I'm setting the threshold for *active* to be greater than 1 because, who knows, maybe random fragments aligned to genes and this might help filter out some noise.

In [ ]:
count_and_tss['active'] = (count_and_tss['count'] > 1)
count_and_tss.head()

Since `True` is equal to `1`, and `False` is equal to `0`, we can determine the number of active genes by adding together the values in `active`.

In [ ]:
sum(count_and_tss['active'])

---

# Visualizing the difference in ChIP-seq data for Active and Not Active Genes

At this point, before we try to apply Naive Bayes to the data, we should graph the average counts for each ChIP-seq enrichment to see if there are differences between *active* and *inactive* genes. So, first, let's calculate the average the number of enriched sites for each ChIP-seq dataset, keeping separate tallies for *active* and *inactive* genes.

In [ ]:
## NOTE: The len() function tells us how many rows are in a dataframe. alternatively, we could use shape[0]

## The general idea in the code below for calculating the average number of
## of enriched Histone modifications is:
##  1) We select only the rows in count_and_tss where the value in the column 
##     'active' is True (or False) and add up the number of enriched sites.
##  2) Divide by the number number of rows in count_and_tss where the value in 
##     'active' is True (or False)
##
## For example, to cacluate the average number of enriched h3k27me3 sites 
## near the TSSs of active genes, we select the 'active' genes with this...
##
##   count_and_tss.loc[count_and_tss['active'] == True]
##
## ...of the rows where 'active' == True, we then specify which column 
## we are interested in, h3k27me3, by adding ['h3k27me3'] like this...
##
##   count_and_tss.loc[count_and_tss['active'] == True]['h3k27me3']
## 
## ...we then add up all the values in the h3k27me3 column...
##
##   sum(count_and_tss.loc[count_and_tss['active'] == True]['h3k27me3'])
##
## ...we then divide divide by the number of active genes with this...
##
##   / len(count_and_tss.loc[count_and_tss['active'] == True])
##
## NOTE: Because the full line of code would be really long, we split it into 
## to lines with the '\' character, which tells python to continue with the code on
## the next line

## h3k27me3
h3k27me3_active = sum(count_and_tss.loc[count_and_tss['active'] == True]['h3k27me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == True])
h3k27me3_inactive = sum(count_and_tss.loc[count_and_tss['active'] == False]['h3k27me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == False])

## h3k36me3
h3k36me3_active = sum(count_and_tss.loc[count_and_tss['active'] == True]['h3k36me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == True])
h3k36me3_inactive = sum(count_and_tss.loc[count_and_tss['active'] == False]['h3k36me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == False])

## h3k4me1
h3k4me1_active = sum(count_and_tss.loc[count_and_tss['active'] == True]['h3k4me1']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == True])
h3k4me1_inactive = sum(count_and_tss.loc[count_and_tss['active'] == False]['h3k4me1']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == False])

## h3k4me3
h3k4me3_active = sum(count_and_tss.loc[count_and_tss['active'] == True]['h3k4me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == True])
h3k4me3_inactive = sum(count_and_tss.loc[count_and_tss['active'] == False]['h3k4me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == False])

## h3k9me3
h3k9me3_active = sum(count_and_tss.loc[count_and_tss['active'] == True]['h3k9me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == True])
h3k9me3_inactive = sum(count_and_tss.loc[count_and_tss['active'] == False]['h3k9me3']) / \
                    len(count_and_tss.loc[count_and_tss['active'] == False])

Now let's combine all of the data into a new DataFrame that we can use as input to a graphing function...

In [ ]:
chip_counts_data = {'chip': ["h3k27me3", "h3k27me3",
                             "h3k36me3", "h3k36me3",
                             "h3k4me1", "h3k4me1",
                             "h3k4me3", "h3k4me3",
                             "h3k9me3", "h3k9me3"],
                    'status': ["active", "inactive",
                               "active", "inactive",
                               "active", "inactive",
                               "active", "inactive",
                               "active", "inactive"],
                    'avg_count': [h3k27me3_active, h3k27me3_inactive, 
                              h3k36me3_active, h3k36me3_inactive, 
                              h3k4me1_active, h3k4me1_inactive,
                              h3k4me3_active, h3k4me3_inactive,
                              h3k9me3_active, h3k9me3_inactive]}
chip_counts_df = pd.DataFrame(data=chip_counts_data)
chip_counts_df

And lastly, let's draw a bargraph that compares the total counts for each ChIP-seq dataset between *active* and *inactive* genes.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6)) ## this is how you make a seaborn graph wider
sns.barplot(x="chip", y="avg_count", hue="status", data=chip_counts_df)

Generally speaking, it looks like, on average, there are more enriched regions of **H3K27me3** around the TSSs of *inactive* genes, and more enriched regions of **H3K36me3**, **H3K4me1** and **H3K4me3** around the TSSs of *active* genes. **H3K9me3** does not look very different between *active* and *inactive* genes.

---

# Applying Naive Bayes to our data

First, let's split the data into independent and dependent variables. Traditionally, the independent variables (in this example, the ChIP-seq data are the independent variables because we are using them to make predictions) are stored in a variable called `X` (upper case **X**), and the dependent variable (`active`, because we are predicting this value) is stored in a variable called `y` (lower case **y**). Let's start by creating `X`.

In [ ]:
X = count_and_tss[['gene_id_stable', 'h3k27me3', 'h3k36me3', 'h3k4me1', 'h3k4me3', 'h3k9me3']].copy()
X.set_index('gene_id_stable', inplace=True) ## this makes gene_id_stable the row names, which 
                                            ## we can be useful if we want to look at specific genes later on
X.head()

Now let's create `y`.

In [ ]:
y = count_and_tss['active'].copy()
y.head()

Now that we have `X` and `y`, we can split them into training and testing datasets.

In [ ]:
## split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Now the part we've been waiting for. We train the Naive Bayes model, test it and draw a [confusion matrix](https://youtu.be/Kdsp6soqA7o). Just 3* lines of code!!! 

\*If you don't count the one line that just makes the confusion matrix larger and easier to read.

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
fig, ax = plt.subplots(figsize=(8, 8)) ## this just makes the confusion matrix larger.
plot_confusion_matrix(clf, X_test, y_test, display_labels=['Not Active', 'Active'], ax=ax)

So, of the... 

In [ ]:
len(y_test)

...genes in the testing dataset, we correctly classified...

In [ ]:
predicted = clf.predict(X_test)
sum(predicted == y_test)

...genes, or 82%, which is pretty good for a very simple model and the possibility that we are not always using the correct TSS for each gene.

## DOUBLE BAM!!!

Now, I wouldn't be surprised if some of you are thinking, "What's the big deal? We already knew which genes were active an inactive from the RNA-seq data." And that is a very valid thing to think. However, consider that the method and approach that we just used can be applied to all kinds of situations in genomics. For example, if we only have a handful of known enhancer or promoter regions, we could use this method to identify more of them. And if we wanted to use a fancier machine learning method, like [Support Vector Machines](https://youtu.be/efR1C6CvhmE) or [Neural Networks](https://youtu.be/CqOfi41LfDw), we only need to make minor changes to those three lines of code that trained and tested the Naive Bayes classifier. So, even though the discovery we made in this Notebook isn't ground breaking, the method can lead you to so many more discoveries, and I think that is pretty amazing!!!!

# TRIPLE BAM!!!